In [3]:
import cv2
from matplotlib import pyplot as plt
import numpy as np 
import imutils
import easyocr
from pathlib import Path
import pandas as pd

In [4]:
folder = Path("D:\\Jiw\\work\\01-school-Work\\01-classroom-Work\\02-SciProj\\Software\\id-injector\\Sci-project-m5\\data")
original_excel = pd.read_excel("แบบคัดกรองซึมเศร้า1266-480Export.xlsx")

In [5]:
global score_table
score_table = pd.DataFrame(original_excel[['ID', 'Pt']])

print(score_table)

      ID  Pt
0    517  19
1    518  18
2    519  27
3    520  34
4    521  15
..   ...  ..
475   47   3
476   48   9
477   50  14
478   51   9
479   52   6

[480 rows x 2 columns]


In [6]:
i = score_table['Pt'][score_table['ID']==2].index
# print(i)
print(score_table.loc[i[0], 'Pt'])

15


In [7]:
def labelClass(ID):
    D = "Depressed"
    noD = "No-Depressed"

    i = score_table['Pt'][score_table['ID']==ID].index
    score = score_table.loc[i[0], 'Pt']
    
    if score >= 15:
        return D
    else:
        return noD

In [8]:
def cropped(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bfilter = cv2.bilateralFilter(gray, 15, 20, 20)
    edged = cv2.Canny(bfilter, 30, 200)

    keypoint = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoint)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    mask = np.zeros(gray.shape, np.uint8)
    location = None

    for contour in contours:
        # print(i)
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            max_x = max(approx[:,:,1])
            min_x = min(approx[:,:,1])
            max_y = max(approx[:,:,0])
            min_y = min(approx[:,:,0])

            x_lenght = max_x-min_x
            y_lenght = max_y-min_y

            if x_lenght > 500 and y_lenght > 500:
                location = approx
                break
        
    if location is not None:
        new_image = cv2.drawContours(mask, [location], 0, 255, -1)
        new_image = cv2.bitwise_and(img, img, mask=mask)

        (x, y) = np.where(mask==255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        cropped_image = gray[x1:x2+1, y1:y2+1]
        return 1, cropped_image

    return 0, gray

In [9]:
def detectID(path):
    result = []
    img = cv2.imread(path)
    img = img[1600: , 100:500]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    reader = easyocr.Reader(['en'])
    result = reader.readtext(gray)

    return result

In [10]:
def denoise(img):
    lower = 0  
    upper = 240 

    mask = cv2.inRange(img, lower, upper)

    result = cv2.bitwise_and(img, img, mask=mask)
    # plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    # cv2.imwrite('test-inrange-denoising.jpg', result)
    return result

In [11]:
i = 0

for jpg_file_path in list(folder.rglob('*.jpg')):
    result = detectID(str(jpg_file_path))
    print(result)

    original_img = cv2.imread(str(jpg_file_path))
    check, cropped_img = cropped(original_img)
    result_img = denoise(cropped_img)

    if len(result) > 0:
        Id = result[0][1].rjust(3, '0')
        label = labelClass(int(result[0][1]))
        if check:
            cv2.imwrite(f'data_label\\{label}\\{Id}.jpg', result_img)
        else:
            cv2.imwrite(f'data_label\\{label}\\incomplete-{Id}.jpg', result_img)
    else:
        cv2.imwrite(f'data_label\\noID-{i}.jpg', result_img)
        i += 1
        print("no Id path : ", jpg_file_path)


[([[143, 27], [163, 27], [163, 51], [143, 51]], '2', 0.9507646819409813)] [([[146, 24], [182, 24], [182, 54], [146, 54]], '15', 0.9999908119993876)] [([[145, 27], [165, 27], [165, 51], [145, 51]], '3', 0.4085314043053443)] [] no Id path :  D:\Jiw\work\01-school-Work\01-classroom-Work\02-SciProj\Software\id-injector\Sci-project-m5\data\1-1\1-1-drawing_page-0004.jpg
[([[148, 22], [184, 22], [184, 50], [148, 50]], '27', 1.0)] [([[148, 22], [186, 22], [186, 52], [148, 52]], '13', 0.9999983984201323)] [([[151, 25], [169, 25], [169, 47], [151, 47]], '5', 0.9999980926522767)] [([[150, 20], [186, 20], [186, 50], [150, 50]], '16', 0.9123437853251184)] [([[148, 20], [186, 20], [186, 50], [148, 50]], '22', 0.9999996628252286)] [([[148, 22], [186, 22], [186, 50], [148, 50]], '10', 0.9999982298328214)] [([[150, 20], [186, 20], [186, 50], [150, 50]], '12', 0.9995410167520867)] [([[148, 18], [184, 18], [184, 48], [148, 48]], '19', 0.9999981455391691)] [([[150, 20], [184, 20], [184, 46], [150, 46]], '

KeyboardInterrupt: 